In [ ]:
# Makes spark stop logging so much
from helpers import *
!export SPARK_CONF_DIR = "$HOME/spark_configuration"


spark = spark_init()


In [ ]:
# Read data
history = (
    spark.read.csv('sample_history.csv', sep='|', header=True)
    .toDF("kind", "text", "url")
)

search_history=prepare_data(history)



In [ ]:

pipeline = classifier_pipeline()


In [ ]:

# Work fun classifier
work_fun_samples = [
    # work
    ['how to make a python3 spark dataframe?', 'work'],
    ['swift objective-c inter operation', 'work'],
    ['bootstrapping deployment pipeline', 'work'],
    ['aws route 53 domain configuration', 'work'],
    # fun
    ['where to go out and eat with friends?', 'fun'],
    ['Best things to bring to a beach party', 'fun'],
    ['Were to buy a sex doll?', 'fun'],
    ['Is not yawning in response a sign of a maniac', 'fun'],
]
work_fun_frame = spark.createDataFrame(
    work_fun_samples).toDF("text", "label")
work_fun_classifier = pipeline.fit(work_fun_frame)


In [ ]:
# Running work fun classifier
work_fun_labeled_pretty = run_classifier(
    work_fun_classifier, search_history, 'work')

nonwork_searches = work_fun_labeled_pretty.filter(
    work_fun_labeled_pretty.predicted_label != 'work')
print(f'Non work related searches: {nonwork_searches.count()}')

work_fun_labeled_pretty_sampled = work_fun_labeled_pretty.sample(
    0.1).orderBy(work_fun_labeled_pretty.work_score)
work_fun_labeled_pretty_sampled.show(1000, False)
